### In this task we will classify 28x28 images of handwritten digits using xgboost
---

In [1]:
import os
import cv2
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [2]:
# Function for loading dataset
def load_dataset(path):
    encoder = LabelEncoder()
    labels = []
    images = []

    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        for file in os.listdir(folder_path):
            labels.append(folder)
            img = cv2.imread(os.path.join(folder_path, file), cv2.IMREAD_GRAYSCALE)
            img = img.flatten() / 255.0
            images.append(img)

    labels = encoder.fit_transform(labels)
    return np.array(images), np.array(labels)

In [3]:
# Load data
X_train, y_train = load_dataset(path="data/train")
X_test, y_test = load_dataset(path="data/test")

# Shuffle data
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

In [4]:
# Create pipeline and train it
from xgboost import XGBClassifier

pipeline = Pipeline([
    ("dim-red", PCA(n_components=0.95, svd_solver="full")),
    ("model", XGBClassifier()),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('dim-red', PCA(n_components=0.95, svd_solver='full')),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [5]:
# Evaluate Model on unseen data
y_pred = pipeline.predict(X_test)
print("Accuracy:" , accuracy_score(y_test, y_pred))
print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9626
Confusion matrix:
[[ 961    0    3    1    0    4    9    2    0    0]
 [   0 1120    2    3    1    2    2    0    5    0]
 [   5    0  994    6    5    2    3    5   11    1]
 [   1    0    6  974    1    8    0    6   11    3]
 [   0    1    3    0  947    0    5    0    2   24]
 [   3    1    0   14    4  854    5    1    9    1]
 [   7    3    0    1    3    7  934    0    3    0]
 [   2    6   20    4    4    1    0  965    5   21]
 [   3    0    7   10    8   11    3    3  921    8]
 [   3    6    3    6   15    6    0    6    8  956]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.96      0.96      0.96      1032
           3       0.96      0.96      0.96      1010
           4       0.96      0.96      0.96       982
           5       0.95      0.96      0.96       892
           6       0.97      0.9

## Notes:
---
During this project I made the following observations:
1. PCA even with a high `n_component` of `0.95` (keeps 95% relevance) the model performs well and the run time is drastically reduced
2. Xgboost is a good alternative to simple image classification tasks
3. Shuffling data is important and unshuffled data can compromise the model accuracy on some classes or make the model overfit

### Note: Ignore this cell. This is to organize projects in the README.MD
```json
{
"Tags": ["Image Classification", "Boosting", "Computer Vision", "Dimensionality Reduction"]
}
```